In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, DateType
from pyspark.sql import functions as F

# Khởi tạo Spark session
spark = SparkSession.builder.appName('Laliga').getOrCreate()

# Định nghĩa schema cho tệp CSV
schema = StructType([
    StructField("FTR", StringType(), True),
    StructField("AwayTeam", StringType(), True),
    StructField("AST", IntegerType(), True),  # Away Team Shots on Target
    StructField("Date", DateType(), True),  # Away Team Shots on Target
    
])

# Đọc dữ liệu từ thư mục (streaming)
df_stream = spark.readStream.format("csv") \
    .option("header", "true") \
    .schema(schema) \
    .load("DataStream_cleaned/")  # Đảm bảo đây là thư mục chứa tệp


# Lọc các trận đấu mà đội khách thắng (FTR = 'A')
away_wins_df = df_stream.filter(df_stream["FTR"] == "A")

# Nhóm theo đội khách (AwayTeam) và tính tổng số trận thắng và tổng số cú sút trúng đích (AST)
result_df = away_wins_df.groupBy("AwayTeam").agg(
    F.count("FTR").alias("away_wins_count"),  # Số trận thắng trên sân khách
    F.sum("AST").alias("total_shots_on_target")  # Tổng số cú sút trúng đích
)

# Sắp xếp theo số trận thắng giảm dần và lấy 3 đội có số trận thắng cao nhất
top_3_teams_df = result_df.orderBy(F.col("away_wins_count").desc()).limit(3)

# Xuất kết quả ra console
query = top_3_teams_df.writeStream.outputMode("complete").format("console").start()

# Đợi cho đến khi stream dừng
query.awaitTermination()

25/04/21 22:46:08 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/20/3zj4bhnd5gl57qd2cr57vsk40000gn/T/temporary-53cb2dcc-3011-474d-b5f2-5c9be3791691. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/04/21 22:46:08 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
                                                                                

-------------------------------------------
Batch: 0
-------------------------------------------
+--------+---------------+---------------------+
|AwayTeam|away_wins_count|total_shots_on_target|
+--------+---------------+---------------------+
+--------+---------------+---------------------+



-------------------------------------------
Batch: 1
-------------------------------------------
+-----------+---------------+---------------------+
|   AwayTeam|away_wins_count|total_shots_on_target|
+-----------+---------------+---------------------+
|Real Madrid|             11|                   65|
|    Sevilla|              9|                   49|
|  Barcelona|              9|                   60|
+-----------+---------------+---------------------+



ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/anaconda3/envs/spark_env/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/anaconda3/envs/spark_env/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/anaconda3/envs/spark_env/lib/python3.10/socket.py", line 717, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 